# LR Filtered Dataset

Testing the classifier when using a filtered dataset (selected malware types e.g. virus, trojan, worm)

In [2]:
import pandas as pd
import numpy as np
import math
from IPython.display import display
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import lib.helpers as jcfg_helpers


samples = pd.read_csv('data/pe32_imports_labeled.csv')
samples['date'] = pd.to_datetime(samples['date'], format='%Y/%m/%d')
samples = samples.set_index('date').sort_index()

goodware = samples[samples.malware == 0]
malware = samples[samples.malware == 1]

print('Total goodware: {}'.format(len(goodware)))
print('Total malware: {}'.format(len(malware)))

Total goodware: 34509
Total malware: 67873


## Dataset for Trojans

In [2]:
microsoft_class = pd.read_csv('data/microsoft_classification.csv')
microsoft_class.fillna('', inplace=True)
symantec_class = pd.read_csv('data/symantec_classification.csv')
symantec_class.fillna('', inplace=True)
trendmicro_class = pd.read_csv('data/trendmicro_classification.csv')
trendmicro_class.fillna('', inplace=True)

microsoft_filter = microsoft_class[microsoft_class.type == 'trojan']
symantec_filter = symantec_class[symantec_class.prefix == 'trojan']
trendmicro_filter = trendmicro_class[trendmicro_class.prefix == 'troj']

links = pd.concat([microsoft_filter.link, symantec_filter.link, trendmicro_filter.link]).unique()

filtered_malware = samples[samples.link.isin(links)]

print('Total malware: {}'.format(len(filtered_malware)))

Total malware: 41870


## Testing

Creating multiple LR and varying the ratio of malware/goodware in training/validation

In [12]:
for i in np.arange(0.1, 1.1, 0.1):
    print('\t\tDownscale of {:.1f}'.format(i))
    # Training/Validation creation
    training, validation = jcfg_helpers.train_test_split(
    pd.concat([goodware, filtered_malware]),
    validation_size = 0.25,
    balanced = True,
    malware_downscale = i)
    #print('Malware in training: {}\nGoodware in training: {}'.format(len(training[training.malware == 1]),
    #                                                                 len(training[training.malware == 0])))
    #print('Malware in validation: {}\nGoodware in validation: {}'.format(len(validation[validation.malware == 1]),
    #                                                                     len(validation[validation.malware == 0])))
    
    ################
    # Feature Vector
    cv_token_pattern = u'[^;]+'
    min_df = 2
    cv = CountVectorizer(token_pattern=cv_token_pattern, min_df=min_df)
    train_X = cv.fit_transform(training.imports)
    #print('Total features: {}'.format(len(cv.get_feature_names())))
    
    ################
    # Logistic Regression
    train_Y = training.malware
    lr = LogisticRegression().fit(train_X, train_Y)

    ################
    # Scoring
    test_X = cv.transform(validation.imports)
    test_Y = validation.malware
    score = confusion_matrix(test_Y, lr.predict(test_X))
    fp_rate, fn_rate, cc_rate, det_rate = jcfg_helpers.calc_ratios(score)
    print('CC:\t{:.4f}'.format(cc_rate))
    print('DR:\t{:.4f}'.format(det_rate))
    print('FP:\t{:.4f}'.format(fp_rate))
    print('FN:\t{:.4f}'.format(fn_rate))
    malware_only = lr.predict_proba(test_X[np.where(test_Y == 1)])
    goodware_only = lr.predict_proba(test_X[np.where(test_Y == 0)])
    print('\nMalware stats:')
    display(pd.Series(malware_only[:,1]).describe())
    print('\nGoodware stats:')
    display(pd.Series(goodware_only[:,0]).describe())
    print()

		Downscale of 0.1
CC:	0.9150
DR:	0.3082
FP:	0.4426
FN:	0.0661

Malware stats:


count    3.310000e+02
mean     3.633775e-01
std      3.564969e-01
min      2.154350e-07
25%      5.470049e-02
50%      2.420007e-01
75%      6.038979e-01
max      9.999999e-01
dtype: float64


Goodware stats:


count    3318.000000
mean        0.942601
std         0.132651
min         0.024514
25%         0.945300
50%         0.989109
75%         0.999479
max         1.000000
dtype: float64


		Downscale of 0.2
CC:	0.8963
DR:	0.5415
FP:	0.2329
FN:	0.0865

Malware stats:


...

count    3.318000e+03
mean     7.731386e-01
std      2.750734e-01
min      2.318087e-08
25%      4.887798e-01
50%      9.282055e-01
75%      9.980447e-01
max      1.000000e+00
dtype: float64


		Downscale of 1.0
CC:	0.7819
DR:	0.8677
FP:	0.2593
FN:	0.1597

Malware stats:


count    3.318000e+03
mean     7.214340e-01
std      2.671056e-01
min      2.697429e-10
25%      5.520293e-01
50%      8.297241e-01
75%      9.601650e-01
max      1.000000e+00
dtype: float64


Goodware stats:


count    3.318000e+03
mean     7.575619e-01
std      2.871125e-01
min      1.954325e-08
25%      4.479707e-01
50%      9.216381e-01
75%      9.974889e-01
max      1.000000e+00
dtype: float64

## Dataset for Backdoors

In [20]:
microsoft_class = pd.read_csv('data/microsoft_classification.csv')
microsoft_class.fillna('', inplace=True)
symantec_class = pd.read_csv('data/symantec_classification.csv')
symantec_class.fillna('', inplace=True)
trendmicro_class = pd.read_csv('data/trendmicro_classification.csv')
trendmicro_class.fillna('', inplace=True)

microsoft_filter = microsoft_class[microsoft_class.type == 'backdoor']
symantec_filter = symantec_class[symantec_class.prefix == 'backdoor']
trendmicro_filter = trendmicro_class[trendmicro_class.prefix == 'bkdr']

links = pd.concat([microsoft_filter.link, symantec_filter.link, trendmicro_filter.link]).unique()

filtered_malware = samples[samples.link.isin(links)]

print('Total malware: {}'.format(len(filtered_malware)))

Total malware: 13273


## Testing

Creating multiple LR and varying the ratio of malware/goodware in training/validation

In [9]:
for i in np.arange(0.1, 1.1, 0.1):
    print('\t\tDownscale of {:.1f}'.format(i))
    # Training/Validation creation
    training, validation = jcfg_helpers.train_test_split(
    pd.concat([goodware, filtered_malware]),
    validation_size = 0.25,
    balanced = True,
    malware_downscale = i)
    #print('Malware in training: {}\nGoodware in training: {}'.format(len(training[training.malware == 1]),
    #                                                                 len(training[training.malware == 0])))
    #print('Malware in validation: {}\nGoodware in validation: {}'.format(len(validation[validation.malware == 1]),
    #                                                                     len(validation[validation.malware == 0])))
    
    ################
    # Feature Vector
    cv_token_pattern = u'[^;]+'
    min_df = 2
    cv = CountVectorizer(token_pattern=cv_token_pattern, min_df=min_df)
    train_X = cv.fit_transform(training.imports)
    #print('Total features: {}'.format(len(cv.get_feature_names())))
    
    ################
    # Logistic Regression
    train_Y = training.malware
    lr = LogisticRegression().fit(train_X, train_Y)

    ################
    # Scoring
    test_X = cv.transform(validation.imports)
    test_Y = validation.malware
    score = confusion_matrix(test_Y, lr.predict(test_X))
    fp_rate, fn_rate, cc_rate, det_rate = jcfg_helpers.calc_ratios(score)
    print('CC:\t{:.4f}'.format(cc_rate))
    print('DR:\t{:.4f}'.format(det_rate))
    print('FP:\t{:.4f}'.format(fp_rate))
    print('FN:\t{:.4f}'.format(fn_rate))
    malware_only = lr.predict_proba(test_X[np.where(test_Y == 1)])
    goodware_only = lr.predict_proba(test_X[np.where(test_Y == 0)])
    print('\nMalware stats:')
    display(pd.Series(malware_only[:,1]).describe())
    print('\nGoodware stats:')
    display(pd.Series(goodware_only[:,0]).describe())
    print()

		Downscale of 0.1
CC:	0.9150
DR:	0.3082
FP:	0.4426
FN:	0.0661

Malware stats:


count    3.310000e+02
mean     3.633775e-01
std      3.564969e-01
min      2.154350e-07
25%      5.470049e-02
50%      2.420007e-01
75%      6.038979e-01
max      9.999999e-01
dtype: float64


Goodware stats:


count    3318.000000
mean        0.942601
std         0.132651
min         0.024514
25%         0.945300
50%         0.989109
75%         0.999479
max         1.000000
dtype: float64


		Downscale of 0.2
CC:	0.8963
DR:	0.5415
FP:	0.2329
FN:	0.0865

Malware stats:


...

count    3.318000e+03
mean     7.731386e-01
std      2.750734e-01
min      2.318087e-08
25%      4.887798e-01
50%      9.282055e-01
75%      9.980447e-01
max      1.000000e+00
dtype: float64


		Downscale of 1.0
CC:	0.7819
DR:	0.8677
FP:	0.2593
FN:	0.1597

Malware stats:


count    3.318000e+03
mean     7.214340e-01
std      2.671056e-01
min      2.697429e-10
25%      5.520293e-01
50%      8.297241e-01
75%      9.601650e-01
max      1.000000e+00
dtype: float64


Goodware stats:


count    3.318000e+03
mean     7.575619e-01
std      2.871125e-01
min      1.954325e-08
25%      4.479707e-01
50%      9.216381e-01
75%      9.974889e-01
max      1.000000e+00
dtype: float64

## Dataset for Worms

In [15]:
microsoft_class = pd.read_csv('data/microsoft_classification.csv')
microsoft_class.fillna('', inplace=True)
symantec_class = pd.read_csv('data/symantec_classification.csv')
symantec_class.fillna('', inplace=True)
trendmicro_class = pd.read_csv('data/trendmicro_classification.csv')
trendmicro_class.fillna('', inplace=True)

microsoft_filter = microsoft_class[microsoft_class.type == 'worm']
symantec_filter = symantec_class[symantec_class.suffix == 'worm']
trendmicro_filter = trendmicro_class[trendmicro_class.prefix == 'worm']

links = pd.concat([microsoft_filter.link, symantec_filter.link, trendmicro_filter.link]).unique()

filtered_malware = samples[samples.link.isin(links)]

print('Total malware: {}'.format(len(filtered_malware)))

Total malware: 9418


## Testing

Creating multiple LR and varying the ratio of malware/goodware in training/validation

In [16]:
for i in np.arange(0.1, 1.1, 0.1):
    print('\t\tDownscale of {:.1f}'.format(i))
    # Training/Validation creation
    training, validation = jcfg_helpers.train_test_split(
    pd.concat([goodware, filtered_malware]),
    validation_size = 0.25,
    balanced = True,
    malware_downscale = i)
    #print('Malware in training: {}\nGoodware in training: {}'.format(len(training[training.malware == 1]),
    #                                                                 len(training[training.malware == 0])))
    #print('Malware in validation: {}\nGoodware in validation: {}'.format(len(validation[validation.malware == 1]),
    #                                                                     len(validation[validation.malware == 0])))
    
    ################
    # Feature Vector
    cv_token_pattern = u'[^;]+'
    min_df = 2
    cv = CountVectorizer(token_pattern=cv_token_pattern, min_df=min_df)
    train_X = cv.fit_transform(training.imports)
    #print('Total features: {}'.format(len(cv.get_feature_names())))
    
    ################
    # Logistic Regression
    train_Y = training.malware
    lr = LogisticRegression().fit(train_X, train_Y)

    ################
    # Scoring
    test_X = cv.transform(validation.imports)
    test_Y = validation.malware
    score = confusion_matrix(test_Y, lr.predict(test_X))
    fp_rate, fn_rate, cc_rate, det_rate = jcfg_helpers.calc_ratios(score)
    print('CC:\t{:.4f}'.format(cc_rate))
    print('DR:\t{:.4f}'.format(det_rate))
    print('FP:\t{:.4f}'.format(fp_rate))
    print('FN:\t{:.4f}'.format(fn_rate))
    malware_only = lr.predict_proba(test_X[np.where(test_Y == 1)])
    goodware_only = lr.predict_proba(test_X[np.where(test_Y == 0)])
    print('\nMalware stats:')
    display(pd.Series(malware_only[:,1]).describe())
    print('\nGoodware stats:')
    display(pd.Series(goodware_only[:,0]).describe())
    print()

		Downscale of 0.1
CC:	0.9289
DR:	0.4596
FP:	0.3455
FN:	0.0524

Malware stats:


count    235.000000
mean       0.458184
std        0.354555
min        0.000011
25%        0.100127
50%        0.464396
75%        0.795255
max        0.999970
dtype: float64


Goodware stats:


count    2354.000000
mean        0.953813
std         0.126771
min         0.000100
25%         0.971733
50%         0.996608
75%         0.999780
max         1.000000
dtype: float64


		Downscale of 0.2
CC:	0.9359
DR:	0.8043
FP:	0.1906
FN:	0.0390

Malware stats:


...

count    2.354000e+03
mean     8.915860e-01
std      2.229390e-01
min      2.915446e-13
25%      9.152757e-01
50%      9.907384e-01
75%      9.996770e-01
max      1.000000e+00
dtype: float64


		Downscale of 1.0
CC:	0.8717
DR:	0.8309
FP:	0.0953
FN:	0.1563

Malware stats:


count    2.354000e+03
mean     7.933651e-01
std      3.097255e-01
min      3.634418e-09
25%      7.942247e-01
50%      9.336490e-01
75%      9.880253e-01
max      1.000000e+00
dtype: float64


Goodware stats:


count    2.354000e+03
mean     8.842927e-01
std      2.302343e-01
min      1.227907e-13
25%      9.018958e-01
50%      9.897749e-01
75%      9.996078e-01
max      1.000000e+00
dtype: float64

## Dataset for Virus

In [18]:
microsoft_class = pd.read_csv('data/microsoft_classification.csv')
microsoft_class.fillna('', inplace=True)
symantec_class = pd.read_csv('data/symantec_classification.csv')
symantec_class.fillna('', inplace=True)
trendmicro_class = pd.read_csv('data/trendmicro_classification.csv')
trendmicro_class.fillna('', inplace=True)

microsoft_filter = microsoft_class[microsoft_class.type == 'virus']
symantec_filter = symantec_class[symantec_class.prefix == 'virusdoctor']
trendmicro_filter = trendmicro_class[trendmicro_class.family == 'virus']

links = pd.concat([microsoft_filter.link, symantec_filter.link, trendmicro_filter.link]).unique()

filtered_malware = samples[samples.link.isin(links)]

print('Total malware: {}'.format(len(filtered_malware)))

Total malware: 4467


## Testing

Creating multiple LR and varying the ratio of malware/goodware in training/validation

In [19]:
for i in np.arange(0.1, 1.1, 0.1):
    print('\t\tDownscale of {:.1f}'.format(i))
    # Training/Validation creation
    training, validation = jcfg_helpers.train_test_split(
    pd.concat([goodware, filtered_malware]),
    validation_size = 0.25,
    balanced = True,
    malware_downscale = i)
    #print('Malware in training: {}\nGoodware in training: {}'.format(len(training[training.malware == 1]),
    #                                                                 len(training[training.malware == 0])))
    #print('Malware in validation: {}\nGoodware in validation: {}'.format(len(validation[validation.malware == 1]),
    #                                                                     len(validation[validation.malware == 0])))
    
    ################
    # Feature Vector
    cv_token_pattern = u'[^;]+'
    min_df = 2
    cv = CountVectorizer(token_pattern=cv_token_pattern, min_df=min_df)
    train_X = cv.fit_transform(training.imports)
    #print('Total features: {}'.format(len(cv.get_feature_names())))
    
    ################
    # Logistic Regression
    train_Y = training.malware
    lr = LogisticRegression().fit(train_X, train_Y)

    ################
    # Scoring
    test_X = cv.transform(validation.imports)
    test_Y = validation.malware
    score = confusion_matrix(test_Y, lr.predict(test_X))
    fp_rate, fn_rate, cc_rate, det_rate = jcfg_helpers.calc_ratios(score)
    print('CC:\t{:.4f}'.format(cc_rate))
    print('DR:\t{:.4f}'.format(det_rate))
    print('FP:\t{:.4f}'.format(fp_rate))
    print('FN:\t{:.4f}'.format(fn_rate))
    malware_only = lr.predict_proba(test_X[np.where(test_Y == 1)])
    goodware_only = lr.predict_proba(test_X[np.where(test_Y == 0)])
    print('\nMalware stats:')
    display(pd.Series(malware_only[:,1]).describe())
    print('\nGoodware stats:')
    display(pd.Series(goodware_only[:,0]).describe())
    print()

		Downscale of 0.1
CC:	0.8998
DR:	0.2432
FP:	0.5909
FN:	0.0724

Malware stats:


count    111.000000
mean       0.277960
std        0.294748
min        0.000009
25%        0.016794
50%        0.199848
75%        0.466603
max        0.940911
dtype: float64


Goodware stats:


count    1116.000000
mean        0.928899
std         0.151273
min         0.006975
25%         0.945831
50%         0.987215
75%         0.996882
max         1.000000
dtype: float64


		Downscale of 0.2
CC:	0.8656
DR:	0.5202
FP:	0.3862
FN:	0.0930

Malware stats:


...

count    1116.000000
mean        0.766688
std         0.302338
min         0.000190
25%         0.661756
50%         0.924666
75%         0.973224
max         1.000000
dtype: float64


		Downscale of 1.0
CC:	0.7836
DR:	0.7894
FP:	0.2197
FN:	0.2131

Malware stats:


count    1.116000e+03
mean     7.321270e-01
std      3.007564e-01
min      3.605102e-07
25%      5.908487e-01
50%      8.649940e-01
75%      9.676963e-01
max      1.000000e+00
dtype: float64


Goodware stats:


count    1116.000000
mean        0.746229
std         0.310891
min         0.000042
25%         0.616377
50%         0.895339
75%         0.969030
max         1.000000
dtype: float64